# **Đọc dữ liệu**

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
import joblib
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("huytrngquc/hanoi-housing-price-2024")

print("Path to dataset files:", path)

data = pd.read_csv('/kaggle/input/hanoi-housing-price-2024/HN_Houseprice.csv')

In [ ]:
data.head(5)

In [ ]:
data.describe()

# **Xử lý dữ liệu**

**1. Xử lý thông tin : Commune (Xã/Phường), District (Quận/Huyện), PostType (Loại hình)**

In [ ]:
data = data[data['PostingDate'] != '0']
data = data[data['Price'] != 'Thỏa thuận']
# Tạo ra để check thay đổi dữ liệu
data2 = data[data['PostingDate'] != '0']
data2 = data[data['Price'] != 'Thỏa thuận']
data.head(5)

In [ ]:
# Hàm trích xuất từ "Phường" hoặc "Xã"
import unicodedata

def extract_commune_clean(row):
    district = row['District']
    address = row['Address']
    
    # Kiểm tra nếu district có trong address
    if district in address:
        # Lấy phần trước giá trị của District trong Address
        parts = address.split(f", {district}")
        
        if len(parts) > 0:
            # Lấy phần tử cuối cùng trước District
            before_district = parts[0].strip()
            
            # Loại bỏ các từ "Phường", "Xã", "Thị trấn" (cả chữ hoa và chữ thường)
            commune = before_district.split(",")[-1].strip()
            commune = commune.replace("Phường ", "").replace("Xã ", "").replace("Thị trấn ", "").replace("phường ", "").replace("xã ", "").replace("thị trấn ", "")
            
            # Nếu không tìm thấy giá trị hợp lệ, lấy 2 từ cuối cùng
            if not commune:
                words = before_district.split()
                if len(words) >= 2:
                    commune = f"{words[-2]} {words[-1]}"  # Lấy hai từ cuối cùng

            # Loại bỏ các tỉnh/thành phố lớn (nếu cần)
            if commune in ['Hà Nội', 'Hồ Chí Minh', 'Đà Nẵng', 'Hải Phòng']:  # Có thể thêm các tỉnh khác
                return None 
            
            # Chuẩn hóa Unicode và chuẩn hóa chữ
            commune = unicodedata.normalize('NFC', commune)
            return commune.lower().title()
    
    return None

# Hàm kiểm tra Province trong Address và lọc dữ liệu chỉ giữ lại "Hà Nội"
def filter_by_province(df, province="Hà Nội"):
    # Bỏ dấu '.' nếu có ở cuối địa chỉ
    df['Address'] = df['Address'].apply(lambda x: x.rstrip('.') if isinstance(x, str) else x)
    # Lấy tỉnh/thành phố từ địa chỉ (từ sau dấu phẩy cuối cùng)
    df['Province'] = df['Address'].apply(lambda x: x.split(",")[-1].strip())
    # Lọc các dòng có Province là "Hà Nội"
    filtered_data = df[df['Province'] == province].drop(columns=['Province'])
    return filtered_data
    
# Áp dụng lọc các tin đăng bán ở Hà Nội
data = filter_by_province(data)

# Áp dụng hàm để cập nhật cột Commune
data['Commune'] = data.apply(extract_commune_clean, axis=1)

# Đưa cột 'Commune' lên trước cột 'District'
columns = list(data.columns)
columns.remove("Commune")
columns.insert(columns.index("District"), "Commune")

data = data[columns]
data.head()

In [ ]:
# # Lọc dữ liệu với giá trị chứa 'hàng bồ' trong cột Commune (không phân biệt hoa thường)
# filtered_data = data[data['Commune'].str.contains('hàng bồ', case=False, na=False)]

# # Đếm tần suất xuất hiện của từng giá trị trong DataFrame lọc
# a = filtered_data.value_counts()
# a = pd.DataFrame(a)
# a

In [ ]:
# b = data['Commune'].value_counts()
# b

In [ ]:
#Check thay đổi dữ liệu
data3 = data2[~data2['Title'].isin(data['Title'])]
data3.head()

In [ ]:
# Hàm lọc và thay thế giá trị trong cột 'PostType' với các từ khóa tương ứng
def filter_and_replace_posttype(row):
    post_type = str(row['PostType']).lower().strip()  
    if any(keyword in post_type for keyword in ['đất', 'mảnh đất']):
        return 'Đất'
    elif any(keyword in post_type for keyword in ['chung cư', 'căn hộ']):
        return 'Chung cư'
    elif any(keyword in post_type for keyword in ['nhà', 'biệt thự']):
        return 'Nhà'
    return None

# Thay thế dữ liệu trong cột 'PostType' và lọc ra các dòng có PostType chứa các từ khóa
data['PostType'] = data.apply(filter_and_replace_posttype, axis=1)

# Lọc ra các dòng có 'PostType' không phải None
data = data[data['PostType'].notna()]
data.head()

In [ ]:
data.info()

In [ ]:
data.shape

**2. Xử lý dữ liệu trùng lặp**

In [ ]:
duplicate_titles = data.value_counts()
# Lọc ra các giá trị trùng lặp (xuất hiện hơn 1 lần)
duplicated_records = duplicate_titles[duplicate_titles > 1]
# Hiển thị kết quả
print("Các giá trị trùng lặp và số lần lặp:")
duplicated_records.count()

In [ ]:
duplicated_records.to_csv('lap.csv', encoding='utf-8-sig')

In [ ]:
duplicate_titles = data.value_counts()
# Lọc ra các giá trị trùng lặp (xuất hiện hơn 1 lần)
duplicated_records = duplicate_titles[duplicate_titles > 1]
# Hiển thị kết quả
print("Các giá trị trùng lặp và số lần lặp:")
duplicated_records.count()
data = data.drop_duplicates()
data = data.reset_index(drop=True)
print(f"Sau khi loại bỏ các giá trị duplicate, còn lại {len(data)} bản ghi.")
data.to_csv('data.csv', encoding='utf-8-sig')
data.head(3)

In [ ]:
data.duplicated().sum()

**3. Kiểm tra số lượng dữ liệu mỗi loại PostType**

In [ ]:
land_titles = data[
    (data['PostType'].str.contains('đất', case=False, na=False))
]
print(f"Có {len(land_titles)} tin bán đất:")
land_titles.head(5)

In [ ]:
house_titles = data[
    (data['PostType'].str.contains('nhà', case=False, na=False))
]
print(f"Có {len(house_titles)} tin bán nhà:")
house_titles.head(5)

In [ ]:
apartment_titles = data[
    (data['PostType'].str.contains('chung cư', case=False, na=False))
]
print(f"Có {len(apartment_titles)} tin bán chung cư:")
apartment_titles.head(5)

**3. Cột Area**

59 m²	-> 59

In [ ]:
data.head(5)

In [ ]:
# Bước 1: Loại bỏ dấu phân cách hàng nghìn (dấu .)
data['Area'] = data['Area'].str.replace(r'\.(?=\d{3})', '', regex=True)

# Bước 2: Thay dấu phẩy (,) bằng dấu chấm (.)
data['Area'] = data['Area'].str.replace(',', '.')

# Bước 3: Loại bỏ các ký tự không phải số và không phải dấu chấm
data['Area'] = data['Area'].str.replace(r'[^\d.]', '', regex=True).astype(float)

data.head(5)

**4. Cột Bedrooms, Toilets, Floors, Width_metters & Entrancewidth**

x phòng -> x

x tầng -> x

x m -> x

In [ ]:
data['Bedrooms'] = data['Bedrooms'].str.replace(' phòng', '').astype(int)
data['Bathrooms'] = data['Bathrooms'].str.replace(' phòng', '').astype(int)
data['Floors'] = data['Floors'].str.replace(' tầng', '').astype(int)
data['Width_meters'] = data['Width_meters'].str.replace(',', '.').str.replace(' m', '').astype(float)
data['Entrancewidth'] = data['Entrancewidth'].str.replace(',', '.').str.replace(' m', '').astype(float)

**5. Chuyển cột PostingDate thành dạng DateTime**

In [ ]:
data['PostingDate'] = pd.to_datetime(data['PostingDate'], format='%d/%m/%Y')
data = data.sort_values(by='PostingDate', ascending = False)
data = data.reset_index(drop=True)

In [ ]:
data.head(5)

In [ ]:
data['PostingDate'].info()

In [ ]:
data.info()

**6. Xử lý dữ liệu cột Price**

In [ ]:
def convert_to_million(row):
    # Đảm bảo giá trị Price là kiểu chuỗi trước khi kiểm tra
    price = str(row['Price'])
    area = row['Area']
    # Loại bỏ dấu phân cách hàng nghìn (dấu .) trong giá trị Price
    price = price.replace('.', '')
    # Xử lý trường hợp 'tỷ/m²'
    if 'tỷ/m²' in price:
        price_value = price.replace(' tỷ/m²', '').replace(',', '.').strip()  # Loại bỏ các ký tự không cần thiết
        try:
            price_in_million = float(price_value) * 1000  # Chuyển từ tỷ/m² sang triệu/m²
        except ValueError:
            price_in_million = 0  # Gán 0 nếu chuyển đổi thất bại
        # Trả về giá trị tính toán dựa trên diện tích
        if isinstance(area, (int, float)) and area != 0:
            return round(price_in_million, 2)  # Trả về giá trị triệu/m² đã chuyển đổi
    # Xử lý trường hợp 'tỷ' thông thường
    elif 'tỷ' in price:
        price_value = price.replace(' tỷ', '').replace(',', '.').strip()
        try:
            price_in_million = float(price_value) * 1000  # Chuyển từ tỷ sang triệu
        except ValueError:
            price_in_million = 0  # Gán 0 nếu chuyển đổi thất bại
    else:
        price_in_million = row['Price']  # Giữ nguyên giá trị nếu không chứa 'tỷ'
    # Kiểm tra nếu giá trị price_in_million là float và area là số hợp lệ
    if isinstance(price_in_million, (float, int)) and isinstance(area, (int, float)) and area != 0:
        return round(price_in_million / area, 2)  # Tính giá trị trên diện tích và làm tròn 2 chữ số
    return price_in_million  # Nếu không chia được, trả về giá trị gốc

# Áp dụng hàm vào cột Price và Area để chuyển đổi và tính toán giá trị
data['Price'] = data.apply(convert_to_million, axis=1)

data.head()

In [ ]:
import re

def extract_number_from_price(price):
    price = str(price)
    
    # Lọc chỉ giữ lại các ký tự số và dấu phân cách thập phân
    cleaned_price = re.sub(r'[^\d.,]', '', price)  # Xóa tất cả ký tự không phải số và dấu phân cách
    
    # Thay dấu phẩy bằng dấu chấm để có thể chuyển thành float
    cleaned_price = cleaned_price.replace(',', '.')
    
    return float(cleaned_price) if cleaned_price else None

data['Price'] = data['Price'].apply(extract_number_from_price)

In [ ]:
data.head(5)

**7. Xử lý cột Legal**

In [ ]:
# Xóa các dấu '=' và ',' trong cột 'Legal'
data['Legal'] = data['Legal'].str.replace('=', '', regex=False)  # Xóa dấu '='
data['Legal'] = data['Legal'].str.replace('+', '', regex=False)  # Xóa dấu '+'
data['Legal'] = data['Legal'].str.replace('-', '', regex=False)  # Xóa dấu '-'


# Hiển thị kết quả
data.head()

In [ ]:
# Hàm lọc và thay thế giá trị trong cột 'Legal' với các từ khóa tương ứng
def filter_and_replace_legal(row):
    post_type = str(row['Legal']).lower().strip()  
    if any(keyword in post_type for keyword in ['có sổ', 'sổ chuẩn', 'sđcc', 'sổ đỏ', 'sổ đổ', 'số đỏ'
                                                , 'sổ đẹp', 'cất két', 'giấy tờ sở hữu'
                                                , 'sổ nở', 'giấy tờ đầy đủ', 'sổ net','sẵn sổ']):
        return 'Sổ đỏ/Sổ hồng'
    elif any(keyword in post_type for keyword in ['vi bằng']):
        return 'Vi bằng'
    elif any(keyword in post_type for keyword in ['hợp đồng mua bán', '0']):
        return 'Hợp đồng mua bán'
    return 'Hợp đồng mua bán'

# Thay thế dữ liệu trong cột 'Legal' 
data['Legal'] = data.apply(filter_and_replace_legal, axis=1)

In [ ]:
Legal_counts = data['Legal'].value_counts()
Legal_counts.to_csv('Legal_counts.csv', encoding='utf-8-sig')

**8. Xử lý cột Interior**

In [ ]:
Interior_counts = data['Interior'].value_counts()
Interior_counts.to_csv('Interior_counts.csv', encoding='utf-8-sig')

In [ ]:
# Hàm lọc và thay thế giá trị trong cột 'Interior' với các từ khóa tương ứng
def filter_and_replace_interior(row):
    post_type = str(row['Interior']).lower().strip()
    
    if (any(keyword in post_type for keyword in ['full', 'đủ', 'nt', 'nội thất đã làm đẹp', 'hoàn thiện']) and
        not any(keyword in post_type for keyword in ['cao cấp', 'xịn', 'tự thiết kế', 'nhập khẩu', 'víp', 'sang trọng', 'châu âu'])):
        return 'Đầy đủ'
    
    elif any(keyword in post_type for keyword in ['cơ bản']) and 'đầy đủ' not in post_type:
        return 'Cơ bản'
    
    elif any(keyword in post_type for keyword in ['cao cấp', 'xịn', 'tự thiết kế', 'nhập khẩu', 'víp', 'sang trọng', 'châu âu']):
        return 'Cao cấp'
    
    elif any(keyword in post_type for keyword in ['thô', '0', 'không']):
        return 'Không có'
    
    return 'Cơ bản'

# Thay thế dữ liệu trong cột 'Interior'
data['Interior'] = data.apply(filter_and_replace_interior, axis=1)

**9. Chuyển giá trị cột Floors (số tầng) của PostType là 'Chung cư' thành 1**

In [ ]:
data.loc[data['PostType'] == 'Chung cư', 'Floors'] = 1

In [ ]:
data.head()

# **Phân tích**

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.shape

In [ ]:
import numpy as np
print(f"Before filtering: {data.shape}")
# Chỉ chọn các cột số và loại bỏ các cột không cần thiết (nếu có)
numerical_cols = data.select_dtypes(include=[np.number]).columns
def remove_outlier_IQR(df, series):
    Q1 = df[series].quantile(0.25)
    Q3 = df[series].quantile(0.75)
    IQR = Q3 - Q1
    # Xác định ngưỡng loại bỏ ngoại lệ
    lower_bound = Q1 - 1.25 * IQR
    upper_bound = Q3 + 1.25 * IQR 
    # Giữ lại các giá trị trong khoảng hợp lệ
    df_final = df[(df[series] >= lower_bound) & (df[series] <= upper_bound)]
    return df_final
# Loại bỏ ngoại lệ cho từng cột số
for column in numerical_cols:
    if data[column].nunique() > 1:  # Chỉ xử lý nếu cột có hơn 1 giá trị duy nhất
        data = remove_outlier_IQR(data, column)
# Reset index sau khi lọc
data.reset_index(drop=True, inplace=True)
print(f"After filtering: {data.shape}")

In [ ]:
# data = data.sort_values(by='Price', ascending = True)
# data.head(5)

In [ ]:
# Đếm số lượng bài đăng theo quận
data['District'].value_counts().head(5)

In [ ]:
# Đếm số lượng từng hướng nhà
data['Direction'].value_counts()

In [ ]:
# Tính giá trung bình theo loại bất động sản
data.groupby('PostType')['Price'].mean()

In [ ]:
import seaborn as sns
# Vẽ boxplot cho phân phối giá theo quận
plt.figure(figsize=(12, 6))
sns.boxplot(data=data, x='District', y='Price')
plt.title('Phân phối giá theo quận')
plt.xticks(rotation=45)
plt.xlabel('Quận')
plt.ylabel('Giá (triệu/m²)')
plt.show()

In [ ]:
#Phân bổ giá nhà
sns.histplot(data['Price'], kde=True)

In [ ]:
# # Danh sách các đặc trưng số để vẽ biểu đồ so với 'Price'
numerical_features = data.select_dtypes(include=['int64', 'float64']).columns
numerical_features = numerical_features.drop('Price', errors='ignore')
for feature in numerical_features:
    plt.figure(figsize=(8, 4))
    plt.scatter(data[feature], data['Price'], alpha=0.5)
    plt.title(f'Scatter Plot: {feature} vs Price')
    plt.xlabel(feature)
    plt.ylabel('Price')
    plt.grid(True)
    plt.show()

In [ ]:
numerical_cols = data.select_dtypes(include=[np.number]).columns
# Tính ma trận tương quan
corr_matrix = data[numerical_cols].corr()
# Vẽ biểu đồ heatmap
plt.figure(figsize=(8, 6)) 
sns.heatmap(corr_matrix,annot=True,fmt='.2f',cmap='coolwarm',linewidths=0.5,cbar=True)
plt.title('Correlation Heatmap', fontsize=16)
plt.xticks(rotation=45, fontsize=10) 
plt.yticks(fontsize=10)
plt.show()

In [ ]:
data.hist(figsize=(20,30))
plt.show()

In [ ]:
import math
# Vẽ biểu đồ phân phối (histogram) với KDE (Kernel Density Estimate) cho mỗi đặc trưng 
# Danh sách các đặc trưng số
numeric_features = data.select_dtypes(include=['int64', 'float64']).columns
n_cols = 3
# Tính số lượng hàng cần thiết để phù hợp với tất cả các đặc trưng số
n_rows = math.ceil(len(numeric_features) / n_cols)
# Tạo grid các subplots với số lượng hàng và cột đã tính toán
# figsize điều chỉnh kích thước tổng thể của figure (chiều rộng, chiều cao)
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(14, n_rows * 5))
# Làm phẳng grid các axes thành một mảng 1D để dễ dàng truy cập
axes = axes.flatten()
# Duyệt qua các đặc trưng số và vẽ histogram cho mỗi đặc trưng
for i, column in enumerate(numeric_features):
    # Vẽ histogram với kernel density estimate (KDE) cho mỗi đặc trưng
    sns.histplot(data=data[column], ax=axes[i], kde=True)
    axes[i].set_title(f'{column} Distribution')
# Xóa các axes không sử dụng
# Nếu số lượng đặc trưng số ít hơn số lượng subplots, loại bỏ các axes thừa
for j in range(len(numeric_features), len(axes)):
    fig.delaxes(axes[j])
# Điều chỉnh layout của các subplots để tránh bị chồng lấn
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(25, 5))
sns.pairplot(data)
plt.show()

In [ ]:
# Phân bổ số lượng tin bán theo từng Quận/Huyện
data['District'].value_counts().plot(kind='bar', figsize=(12,5))
plt.show()

In [ ]:
# Phân bổ giá bán theo từng Quận/Huyện
plt.figure(figsize=(15, 5))
sns.barplot(x = data['District'], y = data['Price'])
plt.xticks( horizontalalignment="center",rotation = 90 )
plt.xlabel("District")
plt.title("District - Price")
plt.ylabel("Price\n")
plt.show()

In [ ]:
# Trung bình giá bán của từng Quận/Huyện
avg_price_by_district = data.groupby('District')['Price'].mean().sort_values()
plt.figure(figsize=(15, 6))
avg_price_by_district.plot(kind='bar')
plt.title('Average House Price by District')
plt.xlabel('District')
plt.ylabel('Average Price')
plt.xticks(rotation=45,fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
# Trung bình giá bán của từng Quận/Huyện
avg_price_by_district = data[data['District'] == 'Nam Từ Liêm'].groupby('Commune')['Price'].mean().sort_values()
plt.figure(figsize=(15, 6))
avg_price_by_district.plot(kind='bar')
plt.title('Average House Price by Commune - Nam Từ Liêm')
plt.xlabel('Nam Từ Liêm')
plt.ylabel('Average Price')
plt.xticks(rotation=45, fontsize=8)
plt.tight_layout()
plt.show()

# **Model**

In [ ]:
data.to_csv('Formatted_data.csv', encoding='utf-8-sig')
data.head()

In [ ]:
data.describe()

In [ ]:
data.fillna(0, inplace=True)
data= data.drop(columns=['Title', 'Address', 'PostingDate'],axis=1)
data.head(2)

In [ ]:
# One-Hot Encoding cho các cột không phải kiểu số
categorical_columns = ['Commune', 'PostType', 'District', 'Direction', 'Legal', 'Interior']
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

In [ ]:
data['Price'] = np.log1p(data['Price'])

In [ ]:
data.head()

In [ ]:
# Chọn các cột đặc trưng và mục tiêu
X = data.loc[:, data.columns != 'Price']
y = data['Price']

In [ ]:
# Chia tập train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import (
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    SGDRegressor,
    BayesianRidge
)
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
    StackingRegressor
)
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

# **Random Forest**

In [ ]:
model_randomforest = RandomForestRegressor(
        n_estimators=150,  # Tăng số lượng cây
        max_depth=15,  # Tăng chiều sâu để học tốt hơn
        min_samples_split=4,  # Tăng min_samples_split
        min_samples_leaf=2,  # Tăng min_samples_leaf để ổn định hơn
        max_features='sqrt',  # Dùng sqrt để giảm overfitting
        random_state=42
    )

model_randomforest.fit(X_train, y_train)

In [ ]:
joblib.dump(model_randomforest, 'model_randomforest.pkl')  # Lưu mô hình sau khi huấn luyện

In [ ]:
y_pred_rf = model_randomforest.predict(X_test)

In [ ]:
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"MAE: {mae_rf}")
print(f"MSE: {mse_rf}")
print(f"RMSE: {rmse_rf}")
print(f"R2 Score: {r2_rf}")

In [ ]:
plt.scatter(y_test,y_pred_rf);
plt.xlabel('Actual');
plt.ylabel('Predicted');

# **Gradient Boosting**

In [ ]:
# Model GradientBoosting
model_gradientboosting = GradientBoostingRegressor(
    n_estimators=800,  # Tăng số lượng cây để giảm sai số
    learning_rate=0.03,  # Giảm learning rate để tăng độ chính xác
    max_depth=10,  # Tăng chiều sâu để mô hình hóa phức tạp hơn
    min_samples_split=4,  # Tăng min_samples_split để giảm overfitting
    min_samples_leaf=2,  # Tăng min_samples_leaf để ổn định hơn
    subsample=0.9,  # Tăng tỷ lệ mẫu để bao quát tốt hơn
    max_features='sqrt',  # Sử dụng sqrt để giảm overfitting
    random_state=42
)
# Huấn luyện mô hình
model_gradientboosting.fit(X_train, y_train)

In [ ]:
joblib.dump(model_gradientboosting, 'model_gradientboosting.pkl')  # Lưu mô hình sau khi huấn luyện

In [ ]:
y_pred_gb = model_gradientboosting.predict(X_test)

In [ ]:
mae_gb = mean_absolute_error(y_test, y_pred_gb)
mse_gb = mean_squared_error(y_test, y_pred_gb)
rmse_gb = np.sqrt(mse_gb)
r2_gb = r2_score(y_test, y_pred_gb)

print(f"MAE: {mae_gb}")
print(f"MSE: {mse_gb}")
print(f"RMSE: {rmse_gb}")
print(f"R2 Score: {r2_gb}")

In [ ]:
plt.scatter(y_test,y_pred_gb);
plt.xlabel('Actual');
plt.ylabel('Predicted');

# **Stacking model**

In [ ]:
# Các base models
base_models = [
    ('gbm', GradientBoostingRegressor(
        n_estimators=800,  # Tăng số lượng cây để giảm sai số
        learning_rate=0.03,  # Giảm learning rate để tăng độ chính xác
        max_depth=10,  # Tăng chiều sâu để mô hình hóa phức tạp hơn
        min_samples_split=4,  # Tăng min_samples_split để giảm overfitting
        min_samples_leaf=2,  # Tăng min_samples_leaf để ổn định hơn
        subsample=0.9,  # Tăng tỷ lệ mẫu để bao quát tốt hơn
        max_features='sqrt',  # Sử dụng sqrt để giảm overfitting
        random_state=42
    )),
    ('rf', RandomForestRegressor(
        n_estimators=150,  # Tăng số lượng cây
        max_depth=15,  # Tăng chiều sâu để học tốt hơn
        min_samples_split=4,  # Tăng min_samples_split
        min_samples_leaf=2,  # Tăng min_samples_leaf để ổn định hơn
        max_features='sqrt',  # Dùng sqrt để giảm overfitting
        random_state=42
    )),
    ('xgb', XGBRegressor(
        n_estimators=500,  # Tăng số lượng cây để cải thiện hiệu suất
        max_depth=10,  # Tăng chiều sâu
        learning_rate=0.03,  # Giảm learning rate
        subsample=0.85,  # Giữ lại nhiều dữ liệu hơn trong mỗi cây
        colsample_bytree=0.8,  # Tăng mức đa dạng của mỗi cây
        random_state=42
    )),
]

# Meta model
meta_model = LinearRegression(
    n_jobs=-1  # Sử dụng tất cả các luồng (cpu/gpu)
)

# Stacking Regressor
model_stacking = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,  # Thực hiện cross-validation để tăng độ chính xác
    n_jobs=-1  # Sử dụng tất cả các luồng (cpu/gpu)
)

# Huấn luyện mô hình
model_stacking.fit(X_train, y_train)

In [ ]:
joblib.dump(model_stacking, 'model_stacking.pkl')  # Lưu mô hình sau khi huấn luyện

In [ ]:
y_pred_st = model_stacking.predict(X_test)

In [ ]:
mae_st = mean_absolute_error(y_test, y_pred_st)
mse_st = mean_squared_error(y_test, y_pred_st)
rmse_st = np.sqrt(mse_st)
r2_st = r2_score(y_test, y_pred_st)

print(f"MAE: {mae_st}")
print(f"MSE: {mse_st}")
print(f"RMSE: {rmse_st}")
print(f"R2 Score: {r2_st}")

In [ ]:
plt.scatter(y_test,y_pred_st)
plt.xlabel('Actual')
plt.ylabel('Predicted')


# **Compare model**

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 6))
# Biểu đồ 1: Random Forest
axes[0].scatter(y_test, y_pred_rf, alpha=0.6, color='purple')
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=1)
axes[0].set_title("Random Forest")
axes[0].set_xlabel("Actual")
axes[0].set_ylabel("Predicted")
# Biểu đồ 2: Dự đoán từ Gradient Boosting
axes[1].scatter(y_test, y_pred_gb, alpha=0.6, color='blue')
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=1)
axes[1].set_title("Gradient Boosting")
axes[1].set_xlabel('Actual')
axes[1].set_ylabel('Predicted')
# Biểu đồ 3: Dự đoán từ Stacking Model
axes[2].scatter(y_test, y_pred_st, alpha=0.6, color='green')
axes[2].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=1)
axes[2].set_title('Stacking Model)')
axes[2].set_xlabel('Actual')
axes[2].set_ylabel('Predicted')
# Điều chỉnh khoảng cách giữa các biểu đồ
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import joblib

# Dữ liệu mới để dự đoán
new_data = {
    'Area': [47.7],
    'Bedrooms': [0],
    'Bathrooms': [0],
    'Commune': ['Thạch Bàn'],
    'PostType': ['Đất'],
    'District': ['Long Biên'],
    'Direction': ['Tây - Nam'],
    'Legal': ['Sổ đỏ/Sổ hồng'],
    'Interior': ['Không có'],
    'Width_meters': [3.7],
    'Floors': [0],
    'Entrancewidth': [3.5]
}

new_df = pd.DataFrame(new_data)
categorical_columns = ['Commune', 'PostType', 'District', 'Direction', 'Legal', 'Interior']
# Mã hóa dữ liệu mới bằng cách sử dụng cùng một cột phân loại
new_df = pd.get_dummies(new_df, columns=categorical_columns, drop_first=False)

# Đảm bảo rằng tất cả các cột trong dữ liệu mới có mặt trong dữ liệu huấn luyện
missing_cols = set(X.columns) - set(new_df.columns)
for col in missing_cols:
    new_df[col] = 0
new_df = new_df[X.columns]  # Sắp xếp lại các cột theo đúng thứ tự của X
new_df
# # Tải mô hình đã huấn luyện
# model_rr = joblib.load('Houseprice.pkl')
# model_gb = joblib.load('Houseprice.pkl')
# model_st = joblib.load('Houseprice.pkl')
# # Dự đoán giá trị log
# log_prediction = model.predict(new_df)

# # Chuyển đổi từ log về giá trị gốc
# original_prediction = np.expm1(log_prediction)

# # In kết quả dự đoán gốc
# print("Kết quả dự đoán:", original_prediction[0])

In [ ]:
import pandas as pd
import numpy as np
import joblib

# Dữ liệu mới để dự đoán
new_data = {
    'Area': [127],
    'Bedrooms': [0],
    'Bathrooms': [0],
    'Commune': ['Minh Trí'],
    'PostType': ['Đất'],
    'District': ['Sóc Sơn'],
    'Direction': ['0'],
    'Legal': ['Hợp đồng mua bán'],
    'Interior': ['Không có'],
    'Width_meters': [5.48],
    'Floors': [0],
    'Entrancewidth': [0]
}

new_df = pd.DataFrame(new_data)
categorical_columns = ['Commune', 'PostType', 'District', 'Direction', 'Legal', 'Interior']
# Mã hóa dữ liệu mới bằng cách sử dụng cùng một cột phân loại
new_df = pd.get_dummies(new_df, columns=categorical_columns, drop_first=False)

# Đảm bảo rằng tất cả các cột trong dữ liệu mới có mặt trong dữ liệu huấn luyện
missing_cols = set(X.columns) - set(new_df.columns)
for col in missing_cols:
    new_df[col] = 0
new_df = new_df[X.columns]  # Sắp xếp lại các cột theo đúng thứ tự của X
num_columns = new_df.columns.size  # hoặc len(new_df.columns)
print("Số lượng cột:", num_columns)
new_df.to_csv('columns.csv')

# Tải mô hình đã huấn luyện
model_rf = joblib.load('model_randomforest.pkl')
model_gb = joblib.load('model_gradientboosting.pkl')
model_st = joblib.load('model_stacking.pkl')
# Dự đoán giá trị log
log_prediction_rf = model_rf.predict(new_df)
log_prediction_gb = model_gb.predict(new_df)
log_prediction_st = model_st.predict(new_df)

# Chuyển đổi từ log về giá trị gốc
original_prediction_rf = np.expm1(log_prediction_rf)
original_prediction_gb = np.expm1(log_prediction_gb)
original_prediction_st = np.expm1(log_prediction_st)
# In kết quả dự đoán gốc
print("Kết quả dự đoán - RF:", original_prediction_rf[0])
print("Kết quả dự đoán - GB:", original_prediction_gb[0])
print("Kết quả dự đoán - ST:", original_prediction_st[0])


In [ ]:
pred_df=pd.DataFrame({'Actual Value':np.expm1(y_test),'RF':np.expm1(y_pred_rf),'GB':np.expm1(y_pred_gb), 'ST': np.expm1(y_pred_st) })
pred_df.to_csv('compare.csv')
# pred_df = pred_df.sort_values(by='Actual Value', ascending = True)
pred_df

In [ ]:
pred_df.describe()